In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

In [ ]:
rm = pyvisa.ResourceManager()
laser = rm.open_resource('ASRL12::INSTR', baud_rate=9600)

In [5]:
print(rm.list_resources())

('TCPIP0::10.10.60.153::inst0::INSTR', 'TCPIP0::10.10.60.3::inst0::INSTR', 'ASRL9::INSTR', 'ASRL12::INSTR')


In [16]:
REG_Nop=0x00
REG_Mfgr=0x02
REG_Model=0x03
REG_Serial=0x04
REG_Release=0x06
REG_Gencfg=0x08
REG_AeaEar=0x0B
REG_Iocap=0x0D
REG_Ear=0x10
REG_Dlconfig=0x14
REG_Dlstatus=0x15
REG_Channel=0x30
REG_Power=0x31
REG_Resena=0x32
REG_Grid=0x34
REG_Fcf1=0x35
REG_Fcf2=0x36
REG_LF1=0x40
REG_LF2=0x41
REG_Oop=0x42
REG_Opsl=0x50
REG_Opsh=0x51
REG_Lfl1=0x52
REG_Lfl2=0x53
REG_Lfh1=0x54
REG_Lfh2=0x55
REG_Currents=0x57
REG_Temps=0x58
REG_Ftf=0x62
REG_Mode=0x90
REG_PW=0xE0
REG_Csweepsena=0xE5
REG_Csweepamp=0xE4
REG_Cscanamp=0xE4
REG_Cscanon=0xE5
REG_Csweepon=0xE5
REG_Csweepoffset=0xE6
REG_Cscanoffset=0xE6
REG_Cscansled=0xF0
REG_Cscanf1=0xF1
REG_Cscanf2=0xF2
REG_CjumpTHz=0xEA
REG_CjumpGHz=0xEB
REG_CjumpSled=0xEC
REG_Cjumpon=0xED
REG_Cjumpoffset=0xE6

In [17]:
def checksum(byte0,byte1,byte2,byte3):
    bip8 = (byte0 & 0x0f) ^ byte1 ^ byte2 ^ byte3
    bip4 = ((bip8 & 0xf0) >> 4) ^ (bip8 & 0x0f)
    return bip4

### Turn Laser ON/OFF

In [31]:
reg = REG_Resena

data = 256
byte2 = int(data/256)
byte3 = int(data-byte2*256)

x = int(checksum(1,reg,byte2,byte3))*16+1

print(bytes.fromhex('%x' % x))

laser.write_raw(bytes.fromhex('%x' % x))
laser.write_raw(chr(reg))
laser.write_raw(chr(byte2))
laser.write_raw(chr(byte3))

print("[Write] Byte 0:",hex(ord(bytes.fromhex('%x' % x))))
print("[Write] Byte 1:",hex(ord(chr(reg))))
print("[Write] Byte 2:",hex(ord((chr(byte2)))))
print("[Write] Byte 3:",hex(ord((chr(byte3)))))

print("[Read] Byte 0:",hex(ord(laser.read_bytes(1))))
print("[Read] Byte 1:",hex(ord(laser.read_bytes(1))))
b2 = ord(laser.read_bytes(1))
b3 = ord(laser.read_bytes(1))
print("[Read] Byte 2:",hex(b2))
print("[Read] Byte 3:",hex(b3))

print(b2*256+b3)
print('laser on>>>>')

b'\x11'
[Write] Byte 0: 0x11
[Write] Byte 1: 0x32
[Write] Byte 2: 0x1
[Write] Byte 3: 0x0
[Read] Byte 0: 0x44
[Read] Byte 1: 0x32
[Read] Byte 2: 0x1
[Read] Byte 3: 0x0
256
laser on>>>>


### Clean Jump

#### Turn on "Whisper Mode"

In [119]:
reg = REG_Mode

data = 2
byte2 = int(data/256)
byte3 = int(data-byte2*256)

x = int(checksum(1,reg,byte2,byte3))*16+1

print(bytes.fromhex('%x' % x))

laser.write_raw(bytes.fromhex('%x' % x))
laser.write_raw(b'\x90')
laser.write_raw(chr(byte2))
laser.write_raw(chr(byte3))

print("[Write] Byte 0:",hex(ord(bytes.fromhex('%x' % x))))
print("[Write] Byte 1:",hex(ord(chr(reg))))
print("[Write] Byte 2:",hex(ord((chr(byte2)))))
print("[Write] Byte 3:",hex(ord((chr(byte3)))))

print("[Read] Byte 0:",hex(ord(laser.read_bytes(1))))
print("[Read] Byte 1:",hex(ord(laser.read_bytes(1))))
b2 = ord(laser.read_bytes(1))
b3 = ord(laser.read_bytes(1))
print("[Read] Byte 2:",hex(b2))
print("[Read] Byte 3:",hex(b3))

print(b2*256+b3)
print("Laser is in *Whisper Mode*")

b'\xa1'
[Write] Byte 0: 0xa1
[Write] Byte 1: 0x90
[Write] Byte 2: 0x0
[Write] Byte 3: 0x2
[Read] Byte 0: 0x0
[Read] Byte 1: 0x0
[Read] Byte 2: 0xc5
[Read] Byte 3: 0x90
50576
Laser is in *Whisper Mode*


#### Define FCF and GRID (Freq. start point and Freq. Grid)

In [52]:
reg = 0x35

data = 187
byte2 = int(data/256)
byte3 = int(data-byte2*256)

x = int(checksum(1,reg,byte2,byte3))*16+1

print(bytes.fromhex('%x' % x))

laser.write_raw(bytes.fromhex('%x' % x))
laser.write_raw(chr(reg))
laser.write_raw(chr(byte2))
laser.write_raw(chr(byte3).encode("UTF-8"))

print("[Write] Byte 0:",hex(ord(bytes.fromhex('%x' % x))))
print("[Write] Byte 1:",hex(ord(chr(reg))))
print("[Write] Byte 2:",hex(ord((chr(byte2)))))
print("[Write] Byte 3:",hex(ord((chr(byte3)))))

rb_0 = laser.read_bytes(1)
print("[Read] Byte 0:",hex(ord(rb_0)))
print("[Read] Byte 1:",hex(ord(laser.read_bytes(1))))
b2 = ord(laser.read_bytes(1))
b3 = ord(laser.read_bytes(1))
print("[Read] Byte 2:",hex(b2))
print("[Read] Byte 3:",hex(b3))

print(rb_0[1:0] & b'\x03')

b'q'
[Write] Byte 0: 0x71
[Write] Byte 1: 0x35
[Write] Byte 2: 0x0
[Write] Byte 3: 0xbb
[Read] Byte 0: 0x4c
[Read] Byte 1: 0x35
[Read] Byte 2: 0x0
[Read] Byte 3: 0xc2


TypeError: unsupported operand type(s) for &: 'bytes' and 'bytes'

In [ ]:
(int.from_bytes(a, 'big') & int.from_bytes(b, 'big')).to_bytes(max(len(a), len(b)), 'big')

In [27]:
reg = 0x36

data = 103
byte2 = int(data/256)
byte3 = int(data-byte2*256)

x = int(checksum(1,reg,byte2,byte3))*16+1

print(bytes.fromhex('%x' % x))

laser.write_raw(bytes.fromhex('%x' % x))
laser.write_raw(chr(reg))
laser.write_raw(chr(byte2))
laser.write_raw(chr(byte3))

print("[Write] Byte 0:",hex(ord(bytes.fromhex('%x' % x))))
print("[Write] Byte 1:",hex(ord(chr(reg))))
print("[Write] Byte 2:",hex(ord((chr(byte2)))))
print("[Write] Byte 3:",hex(ord((chr(byte3)))))

print("[Read] Byte 0:",hex(ord(laser.read_bytes(1))))
print("[Read] Byte 1:",hex(ord(laser.read_bytes(1))))
b2 = ord(laser.read_bytes(1))
b3 = ord(laser.read_bytes(1))
print("[Read] Byte 2:",hex(b2))
print("[Read] Byte 3:",hex(b3))

print(b2*256+b3)

b'Q'
[Write] Byte 0: 0x51
[Write] Byte 1: 0x36
[Write] Byte 2: 0x0
[Write] Byte 3: 0x67
[Read] Byte 0: 0x4
[Read] Byte 1: 0x36
[Read] Byte 2: 0x0
[Read] Byte 3: 0x67
103


In [12]:
laser.close()

In [107]:
x

1